In [ ]:
# 0값을 검사할 피처명 리스트 객체 설정

zero_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

#pregnancies는 출산 회수를 의미하므로 제외

#전체 데이터 건수
total_count = df['Glucose'].count()

#피처별로 반복 하면서 데이터 값이 0인 데이터 건수 추출하고 퍼센트 계산
for feature in zero_features:
    zero_count = df[df[feature] == 0][feature].count()
    total_count = df[feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, zero_count, 100*zero_count/total_count))

#skinThickness 와 insulin의 0 값은 상당히 많기 때문에 일괄 삭제가 어려우므로 평균값으로 대체

In [ ]:
from sklearn.preprocessing import StandardScaler

# 데이터셋 피처 X, 레이블 y 할당
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# StandardScaler 클래스를 이용하여 속성 데이터 세트에 일괄적으로 스케일링 적용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습/테스트 데이터 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=156, stratify=y)

# 로지스틱 회귀로 학습/예측/평가
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:, 1]

# 평가
get_clf_eval(y_test, pred, pred_proba)

In [ ]:
precision_recall_curve_plot(y_test, pred_proba)

In [ ]:
# 임계값에 따른 평가 수치 출력 함수

from sklearn.preprocessing import Binarizer
pred_proba_c1 = pred_proba.reshape(-1, 1)

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    #thresholds 리스트 객체내의 값을 차례로 iteration하면서 evaluation 수행
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임곗값:', custom_threshold)
        get_clf_eval(y_test, custom_predict, pred_proba_c1)
        print()

In [ ]:
#임곗값 0.3에서 0.5까지 변화시키면서 평가 지표 확인
#정확도를 내리지 않으면서 재현율을 올리는 값

thresholds = [0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.5]
get_eval_by_threshold(y_test, pred_proba_c1, thresholds)